In [1]:
import torch
import random

import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem

from pathlib import Path

from torch.utils.data import Dataset

from torch_geometric.data import Data, Batch

from tqdm.notebook import tqdm_notebook

from torch_geometric.utils import to_networkx

import pickle

import zstandard as zstd
from io import BytesIO


In [2]:
import sys
import os
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
sys.path.append(parent_dir)
from DataPipeline.preprocessing import edge_encoder, node_encoder, get_subgraph_with_terminal_nodes, smiles_to_torch_geometric, process_encode_graph
from DataPipeline.preprocessing import plot_graph
from DataPipeline.dataset import ZincSubgraphDataset, custom_collate

In [3]:
csv_path = Path(".\data") / "rndm_zinc_drugs_clean_3.csv"

zinc_df = pd.read_csv(csv_path)

In [4]:
def remove_iodine_bromine_phosphorus(df):

    df = df[df['smiles'].str.contains('Br') == False]
    df = df[df['smiles'].str.contains('I') == False]
    df = df[df['smiles'].str.contains('P') == False]
    
    return df

In [5]:
"""
preprocessed_graph = []
filtered_df = remove_iodine_bromine_phosphorus(zinc_df)
for row in tqdm(filtered_df.itertuples()):
    smiles = row.smiles
    data =  process_encode_graph(smiles, 'reduced')
    preprocessed_graph.append(data)

    
torch.save(preprocessed_graph, 'preprocessed_graph_no_I_Br_P.pt')
"""

"\npreprocessed_graph = []\nfiltered_df = remove_iodine_bromine_phosphorus(zinc_df)\nfor row in tqdm(filtered_df.itertuples()):\n    smiles = row.smiles\n    data =  process_encode_graph(smiles, 'reduced')\n    preprocessed_graph.append(data)\n\n    \ntorch.save(preprocessed_graph, 'preprocessed_graph_no_I_Br_P.pt')\n"

In [6]:
datapath = Path('..') / 'DataPipeline/data/preprocessed_graph_no_I_Br_P.pt'
predataset = ZincSubgraphDataset(data_path = datapath)
dataloader = torch.utils.data.DataLoader(predataset, batch_size=128, shuffle=True, collate_fn=custom_collate)

Dataset encoded with size 7


In [7]:
def save_graph_batch_zst(graph_batch, file_path):
    with open(file_path, 'wb') as f:
            cctx = zstd.ZstdCompressor()
            compressor = cctx.stream_writer(f)
            torch.save(graph_batch, compressor)
            compressor.flush(zstd.FLUSH_FRAME)


def add_save_subgraph(subgraph, atom_type, encoding_option, graph_in_memory, conversion_mapping, threshold, save_dir : Path, target_atom = None):

    
    save_condition = False

    if target_atom is not None:

        reversed_conversion_mapping = {value:key for key, value in conversion_mapping.items()}
        atom_indice = reversed_conversion_mapping[target_atom]

        if atom_type[atom_indice] > 0.01:
            atom_letter = target_atom
            graph_in_memory[atom_letter].append(subgraph)
            if len(graph_in_memory[atom_letter]) >= threshold:
                save_condition = True
        else:
            return graph_in_memory
    else:
        atom_indice = torch.multinomial(atom_type, 1)

        atom_letter = conversion_mapping[int(atom_indice)]
        graph_in_memory[atom_letter].append(subgraph)
        if len(graph_in_memory[atom_letter]) >= threshold or (((atom_letter == 'Cl') or (atom_letter == 'S') or (atom_letter == 'F')) and (len(graph_in_memory[atom_letter]) >= threshold/10)):
            save_condition = True
    if save_condition:
        #save the graph_list to file
        os.makedirs(save_dir / atom_letter, exist_ok=True)

        # add the number at the end of the file name depending on the previous number of files in the folder

        list_dir = os.listdir(save_dir / atom_letter)
        list_dir = [int(file.split('.')[0].split('_')[-1]) for file in list_dir]
        list_dir.sort()
        if list_dir != []:
            number = list_dir[-1] + 1
        else:
            number = 0

        file_path = save_dir / atom_letter / (str(atom_letter) + str(len(graph_in_memory[atom_letter])) + '_' + str(number) + '.zst')

        # convert graph list in a batch 
        graph_batch = Batch.from_data_list(graph_in_memory[atom_letter])
        # Save the graph_batch using zstandard compression
        save_graph_batch_zst(graph_batch, file_path)       
        
        graph_in_memory[atom_letter] = []
    
    return graph_in_memory

def compute_subgraphs(dataloader, encoding_option, threshold, save_dir : Path, num_processes = 1, target_atom = None):
    if encoding_option == 'all':
        graph_in_memory = {'C':[], 'N':[], 'O':[], 'F': [], 'P' : [], 'S' : [], 'Cl' : [], 'Br' : [], 'I' : [], 'stop' : []}
        conversion_mapping = {0:'C', 1:'N', 2:'O', 3:'S', 4:'F', 5:'P', 6:'Cl', 7:'Br', 8:'I', 9:'stop'}
    elif encoding_option == 'reduced':
        graph_in_memory = {'C':[], 'N':[], 'O':[], 'F': [], 'S' : [], 'Cl' : [], 'stop' : []}
        conversion_mapping = {0:'C', 1:'N', 2:'O', 3:'F', 4:'S', 5:'Cl', 6:'stop'}

    for num in tqdm_notebook(range(num_processes), desc="Processes"):
        dataloader_tqdm = tqdm_notebook(dataloader, desc="Batches")
        for batch_idx, batch in enumerate(dataloader_tqdm):
            for i in range(len(batch[1])):
                atom_type = batch[1][i]
                subgraph = batch[0][i]
                graph_in_memory = add_save_subgraph(subgraph, atom_type, encoding_option, graph_in_memory, conversion_mapping, threshold, save_dir, target_atom = target_atom)

            lengths = ", ".join([f"{key}: {len(value)}" for key, value in graph_in_memory.items()])
            dataloader_tqdm.set_description(f"Batches ({lengths})")
    

In [8]:
compute_subgraphs(dataloader, 'reduced', 100000, Path('..') / 'DataPipeline/data/prepared_dataset', num_processes = 1000, target_atom = None)

Processes:   0%|          | 0/1000 [00:00<?, ?it/s]

Batches:   0%|          | 0/1845 [00:00<?, ?it/s]

Batches:   0%|          | 0/1845 [00:00<?, ?it/s]

Batches:   0%|          | 0/1845 [00:00<?, ?it/s]

Batches:   0%|          | 0/1845 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1845 [00:00<?, ?it/s]

Batches:   0%|          | 0/1845 [00:00<?, ?it/s]

Batches:   0%|          | 0/1845 [00:00<?, ?it/s]

Batches:   0%|          | 0/1845 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
def load_compressed_batch(file_path):
    with open(file_path, 'rb') as f:
        dctx = zstd.ZstdDecompressor()
        decompressor = dctx.decompressobj()
        decompressed_data = decompressor.decompress(f.read())
        decompressed_buffer = BytesIO(decompressed_data)
        graph_batch = torch.load(decompressed_buffer)

    return graph_batch

In [ ]:
file_path = Path('..') / 'DataPipeline/data/prepared_dataset' / 'stop' / 'stop10000_1.zst'

graph_batch = load_compressed_batch(file_path)

In [ ]:
graph = graph_batch.to_data_list()[0]

In [ ]:
#plot molecules of the dataset 
def plot_molecules(df, n_molecules):
    for i in range(n_molecules):
        mol = Chem.MolFromSmiles(df.iloc[i].smiles)
        AllChem.Compute2DCoords(mol)
        display(mol)

In [ ]:
plot_molecules(remove_iodine_bromine_phosphorus(zinc_df)[1], 10)

KeyError: 1